In [2]:
# import necessary libraries
import os, csv
# other options:   import os.system(wget http)   OR   import urllib.urlretrieve (this is an alternative command)

In [3]:
sample = [] # make empty list
with open('/hdir/0/jhaber/Dropbox/projects/Workshops_and_coding/charter_project/charters_microsam.csv', 'r', encoding = 'Windows-1252')\
as csvfile: # open file; the windows-1252 encoding looks weird but works for this
    reader = csv.DictReader(csvfile) # create a reader
    for row in reader: # loop through rows
        sample.append(row) # append each row to the list

In [4]:
# Take a look at the contents of our list called sample--just the first entry
print(sample[1]["SEARCH: NAME ADDRESS"], "\n", sample[1]["URL"], "\n", sample[1])

POLK STATE COLLEGE COLLEGIATE HIGH SCHOOL 3425 WINTER LK RD LAC1200, WINTER HAVEN, FL 
 https://www.polk.edu/lakeland-gateway-to-college-high-school/ 
 {'TR04F': '-2', 'WH12M': '36', 'HI08M': '-2', 'AMALF': '0', 'ADDRESS': '3425 WINTER LK RD LAC1200, WINTER HAVEN, FL', 'HIPKM': '-2', 'HP03M': '-2', 'HP11M': '0', 'CMO/EMO NAME (if managed by a CMO/EMO)': '0', 'SPFEMALE': '2', 'TRUGM': '-2', 'G11OFFRD': '1', 'AM01M': '-2', 'G03OFFRD': '2', 'URL BRK': 'FALSE', 'AM07M': '-2', 'ISPWHITE': 'PS', 'FRELCH': '42', 'G05': '-2', 'MSTREE': '3425 WINTER LAKE RD', 'HPPKM': '-2', 'AM05M': '-2', 'WHUGM': '-2', 'WH08F': '-2', 'TR06F': '-2', 'AM10M': '-2', 'SCHNO': '4186', 'SMEMPUP': '2', 'G12OFFRD': '1', 'LZIP': '33881', 'WHPKF': '-2', 'G07': '-2', 'G07OFFRD': '2', 'MEMBER': '271', 'TR05M': '-2', 'HP10M': '-2', 'G10OFFRD': '2', 'TYPE': '1', 'AS09M': '-2', 'HP05M': '-2', 'WHITE': '164', 'FIPST': '12', 'ASPKM': '-2', 'BL08M': '-2', 'SCHNAM': 'POLK STATE COLLEGE COLLEGIATE HIGH SCHOOL', 'CONAME': 'POLK CO

In [5]:
# turning this into tuples we can use with wget!
# first, make some empty lists
url_list = []
name_list = []
terms_list = []

# now let's fill these lists with content from the sample
for school in sample:
    url_list.append(school["URL"])
    name_list.append(school["SCHNAM"])
    terms_list.append(school["SEARCH: NAME ADDRESS"])

In [6]:
# it's VERY important that these three lists be indexed the same, so let's check:
print(url_list[:3], "\n", name_list[:3], "\n", terms_list[:3])

['https://www.richland2.org/charterhigh/', 'https://www.polk.edu/lakeland-gateway-to-college-high-school/', 'https://www.nhaschools.com/schools/rivercity/Pages/default.aspx'] 
 ['RICHLAND TWO CHARTER HIGH', 'POLK STATE COLLEGE COLLEGIATE HIGH SCHOOL', 'RIVER CITY SCHOLARS CHARTER ACADEMY'] 
 ['RICHLAND TWO CHARTER HIGH 750 OLD CLEMSON RD, COLUMBIA, SC', 'POLK STATE COLLEGE COLLEGIATE HIGH SCHOOL 3425 WINTER LK RD LAC1200, WINTER HAVEN, FL', 'RIVER CITY SCHOLARS CHARTER ACADEMY 944 EVERGREEN ST, GRAND RAPIDS, MI']


Success! Looking at the above, the 3 lists with URLs, names, and search terms, respectively, seem to line up as we would expect: URL[0] corresponds to name[0] corresponds to terms[0], and so on. Now, let's turn these lists into tuples so we can use them for wget. To do this, we use the zip function, which ABSOLUTELY DEPENDS on the consistent indexing we have just established:

In [7]:
tuple_list = list(zip(url_list, name_list))
# Let's check what these tuples look like:
print(tuple_list[:3])
print("\n", tuple_list[1][1].title())

[('https://www.richland2.org/charterhigh/', 'RICHLAND TWO CHARTER HIGH'), ('https://www.polk.edu/lakeland-gateway-to-college-high-school/', 'POLK STATE COLLEGE COLLEGIATE HIGH SCHOOL'), ('https://www.nhaschools.com/schools/rivercity/Pages/default.aspx', 'RIVER CITY SCHOLARS CHARTER ACADEMY')]

 Polk State College Collegiate High School


These look great! Now, to use the os library's wget function, we need to manually change directory just as we would in bash/ UNIX. We also need to track which directory we are in and create a folder in which to store the data for each charter.
#### Students: be sure to change the directory listed in these commands to one that works for you! Otherwise it will throw an error, and the wget command below won't work. ####

To start, we cd into the data directory, and then us getcwd (the equivalent of pwd in bash) to check if it worked:

In [8]:
os.chdir('/hdir/0/jhaber/Documents/projects/charter_data/wget_sept8')
os.getcwd()

'/hdir/0/jhaber/Documents/projects/charter_project/urls_extracted'

Success! Now we've got tuples and we're in the right directory to start with. We are finally ready to run wget!

In [9]:
k=0 # initialize this numerical variable k, which keeps track of which entry in the sample we are on.
for tup in tuple_list:
    k += 1 # Add one to k, so we start with 1 and increase by 1 all the way up to entry # 300
    print("Capturing website data for", (tup[1].title()) + ", which is school #" + str(k), "of 300...")
    # use the tuple to create a name for the folder
    if k < 10: # Add two zeros to the folder name if k is less than 10 (for ease of organizing the output folders)
        dirname = "00" + str(k) + " " + (tup[1].title())
    elif k < 100: # Add one zero if k is less than 100
        dirname = "0" + str(k) + " " + (tup[1].title())
    else: # Add nothing if k>100
        dirname = str(k) + " " + (tup[1].title())
    os.chdir('/hdir/0/jhaber/Documents/projects/charter_data/wget_sept8')  # cd into data directory--this is a/
    # key step because otherwise wget puts the output into whatever folder it was previously in, can get real messy.
    os.makedirs(dirname) # create the folder using the dirname we just made, then change into that directory
    os.chdir(dirname)  # other options to think about for wget: --page-requisites --retry-connrefused --convert-links --wait=3
    os.system('wget --no-parent --show-progress --progress=dot --recursive --level=3 --convert-links --retry-connrefused \
        --random-wait --no-cookies --secure-protocol=auto --no-check-certificate --execute robots=off \
        --user-agent="Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:11.0) Gecko/20100101 Firefox/11.0" \
        --reject .mov,.MOV,.avi,.AVI,.mpg,.MPG,.mpeg,.MPEG,.mp3,.MP3,.mp4,.MP4,.png,.PNG,.gif,.GIF,.jpg,.JPG,\
        .jpeg,.JPEG,.pdf,.PDF,.pdf_,.PDF_,.doc,.DOC,.docx,.DOCX,.xls,.XLS,.xlsx,.XLSX,.csv,.CSV,.ppt,.PPT,.pptx,.PPTX'\
              + ' ' + (tup[0]))

Capturing website data for Richland Two Charter High, which is school #1 of 300...
Capturing website data for Polk State College Collegiate High School, which is school #2 of 300...
Capturing website data for River City Scholars Charter Academy, which is school #3 of 300...
Capturing website data for Detroit Enterprise Academy, which is school #4 of 300...
Capturing website data for Lighthouse Community Sch Inc, which is school #5 of 300...
Capturing website data for Westlake Charter Middle, which is school #6 of 300...
Capturing website data for Van Gogh Charter, which is school #7 of 300...
Capturing website data for Summit Academy Transition High School Dayton, which is school #8 of 300...
Capturing website data for Westchester Academy For International Studies, which is school #9 of 300...
Capturing website data for City Academy, which is school #10 of 300...
Capturing website data for Byrneville Elementary School, Inc., which is school #11 of 300...
Capturing website data for Acac

## Troubleshooting wget: what if there's no directory hierarchy?
Although it's not common practice, some sites have no directory hierarchy, and this makes it difficult to scrape them automatically. For instance, Chinook Montessori Charter School's home URL is 'http://www.k12northstar.org/chinook', and if you go to this site you'll find links to other website pages with URLs that are NOT nested within this home URL, e.g. 'http://www.k12northstar.org/Page/2684' and 'http://www.k12northstar.org/Page/2685'. 

Our problem with directory-less URL structures is that the coding strategy used above really depends on wget's "recursive" option, which scrapes the site you give it as well as all sites included within that. For instance, it would scrape not only www.school.com/ but also www.school.com/cafeteria and www.school.com/about_us. This won't work for sites with no directory hierarchy. It may be impossible to separate out the site pages for only the school of interest if the site directory includes multiple schools at a "flat" level with no hierarchy... But we won't know until we try!

Let's look at coding an automated solution to deal with this problem. Thus far, I've only thought up a painstaking solution that involves feeding wget each site manually. Below is an example of that approach.

In [ ]:
# Grabbing #142--a school that has no directory hierarchy in its website:
os.chdir('/Users/Jaren/Dropbox/projects/Workshops_and_coding/charter_project/\
urls_extracted/142 Chinook Montessori Charter School')

for url in ['http://www.k12northstar.org/chinook', 'http://www.k12northstar.org/site/Default.aspx?PageID=2678', \
'http://www.k12northstar.org/Page/2684', 'http://www.k12northstar.org/Page/2685', \
'http://www.k12northstar.org/Page/2686', 'http://www.k12northstar.org/Page/2683', \
'http://www.k12northstar.org/Page/2704']:
    os.system('wget --no-parent --show-progress --progress=dot --convert-links \
        --no-cookies --secure-protocol=auto --no-check-certificate --execute robots=off \
        --user-agent="Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:11.0) Gecko/20100101 Firefox/11.0" \
        --reject .mov,.mpg,.mpeg,.mp3,.png,.gif,.jpg,.jpeg,.pdf,.pdf_,.doc,.xls,.xlsx,.csv,.ppt,.pptx' + ' ' + url)